In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from pysentani import *
import sentani as sti
survey = find_survey("../data-anonymous")

../data-anonymous/sentani-merged-cleaned-anonymous-2014-11-29.xlsx


In [3]:
survey['access_type'] = survey['village_name'].apply(lambda x: sti.access_map.get(x))

In [4]:
frequency = {'PLN_expenditure':'monthly',
'genset_expenditure':'daily'}

multiplier = {'monthly':1, 'daily':30}

# create new columns with same monthly frequency
for column in frequency.keys():
    # adjust expenditure frequency
    expenditure = survey[column] * multiplier[frequency[column]]
    new_column = column + '_monthly'
    survey[new_column] = expenditure
   
    

pd.set_option('max_rows', 200)
survey[['PLN_expenditure_monthly','genset_expenditure_monthly']]
survey.groupby(['access_type', ])[['PLN_expenditure_monthly',
                                  'genset_expenditure_monthly']].agg(['mean'])

,PLN_expenditure_monthly,genset_expenditure_monthly
,mean,mean
access_type,,
PLN_grid,161688.366446,5950000.000000
PLN_microgrid,NaN,1115789.473684
community_microgrid,34000.000000,4500000.000000
no_access,NaN,1135787.191011


In [5]:
#monthly expenditure divided by 960 rp per kWh (.08 USD per kWh * 12000 rp to USD exchange)
def genset_kwh_conversion(df):
    converted = (df['genset_expenditure_monthly']/(960))
    return converted

survey['genset_kWh'] = survey.apply(genset_kwh_conversion, axis=1)

# kWh multiplied by Lawrence Berkely Labratory estimation of 1100 g CO2 per kWh
def CO2_emission(df1):
    converted = df1['genset_kWh']*1100 
    return converted

survey['CO2_g'] = survey.apply(CO2_emission, axis=1)

# Potential Photovoltaic carbon emissions (g CO2 per kWh), (if genset replaced by PV)
def PV_emission(df2):
    converted = df2['genset_kWh']*100
    return converted
survey['PV_emission'] = survey.apply(PV_emission, axis=1)

# CO2 reduction (co2(g) column - PV_emission column)

#def CO2_reduction(df3):
#    converted = df3(['co2(g)'] - ['PV_emisssion'])
#    return converted
#survey['CO2_reduction'] = survey.apply(CO2_reduction, axis=1)

In [6]:
df = survey.groupby(['access_type',
                'village_name'])[['genset_expenditure_monthly',
                                  'genset_kWh','CO2_g','PV_emission']].agg(['mean']).dropna()


In [7]:
df

genset_expenditure_monthly    genset_kWh  \
                                                       mean          mean   
access_type         village_name                                            
PLN_grid            Hobong                  15000000.000000  15625.000000   
                    Khamayakha               1425000.000000   1484.375000   
PLN_microgrid       Ayapo                    1115789.473684   1162.280702   
community_microgrid Atamali                  4500000.000000   4687.500000   
no_access           Abar                     1200000.000000   1250.000000   
                    Ebunfauw                  988888.888889   1030.092593   
                    Kalio                    1150000.000000   1197.916667   
                    Kampung_Baru             1280000.000000   1333.333333   
                    Khageuw                  1050000.000000   1093.750000   
                    Puai                     1419285.714286   1478.422619   
                    Yoboi                     703452.413793    732.762931   
                    Yokiwa                   1143750.000000   1191.406250   

                                            CO2_g     PV_emission  
                                             mean            mean  
access_type         village_name                                   
PLN_grid            Hobong        17187500.000000  1562500.000000  
                    Khamayakha     1632812.500000   148437.500000  
PLN_microgrid       Ayapo          1278508.771930   116228.070175  
community_microgrid Atamali        5156250.000000   468750.000000  
no_access           Abar           1375000.000000   125000.000000  
                    Ebunfauw       1133101.851852   103009.259259  
                    Kalio          1317708.333333   119791.666667  
                    Kampung_Baru   1466666.666667   133333.333333  
                    Khageuw        1203125.000000   109375.000000  
                    Puai           1626264.880952   147842.261905  
                    Yoboi           806039.224138    73276.293103  
                    Yokiwa         1310546.875000   119140.625000

In [8]:
df.reset_index(inplace=True)

In [9]:
df.head()

,access_type,village_name,genset_expenditure_monthly,genset_kWh,CO2_g,PV_emission
,,,mean,mean,mean,mean
0,PLN_grid,Hobong,15000000.000000,15625.000000,17187500.00000,1562500.000000
1,PLN_grid,Khamayakha,1425000.000000,1484.375000,1632812.50000,148437.500000
2,PLN_microgrid,Ayapo,1115789.473684,1162.280702,1278508.77193,116228.070175
3,community_microgrid,Atamali,4500000.000000,4687.500000,5156250.00000,468750.000000
4,no_access,Abar,1200000.000000,1250.000000,1375000.00000,125000.000000


In [15]:
#df.columns = ['_'.join(col).strip() for col in df.columns.values]


In [16]:
#df.columns = ['access_type', 'vn', 'gem_mean', 'genset_kwh_mean', 'carbon']

In [14]:
#g = sns.FacetGrid(col="access_type", data=df)
#g = (g.map(sns.barplot, "vn", "carbon")
#    .set(plt.xticks(rotation=45)))